# Table Linker Files

Build the files required for Table linker Elasticsearch Index.

You'll need to run the [Wikidata Useful Files](https://github.com/usc-isi-i2/kgtk/blob/master/use-cases/Wikidata%20Useful%20Files.ipynb) prior to running this notebook as files created by that notebook will be used here.

In [3]:
import os
import pandas as pd

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [ ]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = "/Volumes/saggu-ssd/wikidata-2021-10-27"
output_path = "/Volumes/saggu-ssd/wikidata-2021-10-27"
project_name = "table-linker-files"

graph_cache_path = None

In [ ]:
files = [
    "label",
    "alias",
    "item",
    "p279star",
    "claims",
    "isa",
    "isastar",
    "all"
]
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  graph_cache_path=graph_cache_path)

In [ ]:
ck.print_env_variables()

In [ ]:
%%time
if graph_cache_path is None:
    ck.load_files_into_cache()

## Create DWD ISA (Variant of IS A)

In [ ]:
kgtk("""filter -i "$item" -p '; P31, P279, P106, P39 ;' -o "$TEMP"/derived.P31_39_106_279.1.tsv.gz""")

In [ ]:
with open(os.environ['TEMP'] + '/custom-edges.tsv', 'w') as fp:
        fp.write("node1\tlabel\tnode2\n")
        fp.write("Q215627\tdwd_isa\tQ5\n") # person dwd_isa human
        fp.write("Q12737077\tdwd_isa\tQ5\n") # occupation dwd_isa human (perhaps controversial)
        fp.write("Q5\tdwd_isa_\tQ215627\n") # inverse
        fp.write("Q5\tdwd_isa_\tQ12737077\n") # inverse
fp.close()

In [ ]:
kgtk("""cat -i "$TEMP"/derived.P31_39_106_279.1.tsv.gz
        -i "$TEMP"/custom-edges.tsv 
        -o "$OUT"/derived.dwd_isa.tsv.gz""")

In [ ]:
!zcat < "$OUT"/derived.dwd_isa.tsv.gz | wc -l

## Compute TF IDF : Class and Property count files

### Class Counts

In [ ]:
!$kypher  -i p279star -i "$OUT"/derived.dwd_isa.tsv.gz \
    --match 'dwd_isa: (n1)-[]->(class), p279star: (class)-[]->(super_class)' \
    --return 'distinct class as node1, "P31_39_106_279star" as label, super_class as node2' \
    --order-by 'node1, label, node2' \
    / add-id --id-style wikidata \
    -o "$OUT"/derived.P31_39_106_279star.tsv.gz

In [ ]:
!$kypher -i "$OUT"/derived.P31_39_106_279star.tsv.gz --as dwd_isa_star -i "$OUT"/derived.dwd_isa.tsv.gz --as P31_39_106_279 \
    --match 'P31_39_106_279: (n1)-[]->(class), dwd_isa_star: (class)-[]->(super_class)' \
    --return 'distinct super_class as node1, count(distinct n1) as node2, "P31_39_106_279_count" as label' \
    --order-by 'node1, label, node2' \
    -o "$OUT"/derived.dwd.count.tsv.gz

In [ ]:
!$kypher -i dwd_isa_star -i P31_39_106_279 -i "$OUT"/derived.dwd.count.tsv.gz \
    --match 'P31_39_106_279: (n1)-[]->(class), dwd_isa_star: (class)-[]->(super_class), count: (super_class)-[]->(count)' \
    --return 'distinct n1 as node1, "class_count" as label, printf("%s:%s", super_class, count) as node2' \
    --order-by 'node1, label, node2' \
    -o "$TEMP"/dwd_isa_class_count.tsv.gz

In [ ]:
kgtk("""sort -i "$TEMP"/dwd_isa_class_count.tsv.gz 
        -X "--parallel 8 --buffer-size 60%" 
        -o "$TEMP"/dwd_isa_class_count.sorted.tsv.gz""")

In [ ]:
kgtk("""compact -i "$TEMP"/dwd_isa_class_count.sorted.tsv.gz 
        --mode=NONE 
        --columns node1 label 
        --presorted True 
        -o "$OUT"/dwd_isa_class_count.compact.tsv.gz""")

### Property Counts

#### For each property get the number of node1 that it occurs in

In [ ]:
!$kypher -i claims \
    --match '(n1)-[l {label: property}]->()' \
    --return 'distinct property as node1, count(distinct n1) as node2, "nodes_count" as label' \
    -o "$TEMP"/property.count.tsv.gz

#### For each item, list the properties it has

In [ ]:
!$kypher -i claims \
    --match '(n1)-[l {label: property}]->()' \
    --return 'distinct n1 as node1, property as node2, "property" as label' \
    -o "$TEMP"/item.property.tsv.gz

#### Combine the property and the counts into one column

In [ ]:
!$kypher -i "$TEMP"/property.count.tsv.gz -i "$TEMP"/item.property.tsv.gz \
    --match 'count: (property)-[]->(count), item: (n1)-[]->(property)' \
    --return 'distinct n1 as node1, "property_count" as label, printf("%s:%s", property, count) as node2' \
    --order-by 'node1, label, node2' \
    -o "$TEMP"/item.property.count.tsv.gz

#### Put all the property/count pairs in one row for each node

In [ ]:
kgtk("""sort -i "$TEMP"/item.property.count.tsv.gz 
        --sort-command gsort 
        -X "--parallel 8 --buffer-size 60%" 
        -o "$TEMP"/item.property.count.sorted.tsv.gz""")

In [ ]:
kgtk("""compact -i "$TEMP"/item.property.count.sorted.tsv.gz 
        --mode=NONE 
        --columns node1 label 
        --presorted True 
        -o "$OUT"/item.property.count.compact.tsv.gz""")

## Compute Property Values (context) File

### Collecting all the properties that we do not need in the wikibase items

For wikibase-item properties, the context will include the label of the value of the property.

The follwing query creates context information for all wikibase-item properties. We need to trim the set of properties to include only the ones that give us useful values:

- exclude P31, P279 as this info is unlikely to be useful as context
- exclude P793, P47, P1830, P190, P1549 (these properties are likely to give confusing contexts, this is just a small sample)
- exclude all properties that are is-a of P31/P279star of
  - Q19820110 (Wikidata property for property documentation)
  - Q18667213 (Wikidata property about Wikimedia categories)
  - Q51118703 (Wikidata property about Wikimedia templates)
  - Q51118821 (Wikidata property about Wikimedia entities)
  - Q18608359 (Wikidata property to indicate a source)
- exclude properties that are subproperty of (note: some of the following may already be exluded by the above exlusions):
  - P1455 (list of works) 
  - P2354 (has list)

In [ ]:
!wd u P31 P279 P793 P47 P1830 P190 P1549

In [ ]:
if compute_table_linker_files:
    df = pd.DataFrame()
    df['node1'] = ["P31", "P279", "P793", "P47", "P1830", "P190", "P1549"]
    df.to_csv(f"{os.environ['TEMP']}/properties.to.remove.tsv", index=False)

In [ ]:
isa_classes = "Q19820110,Q18667213,Q51118703,Q51118821"

In [ ]:
!wd u Q19820110 Q18667213 Q51118703 Q51118821

Make sure the following files are present in the input path. These files for DWD v2 can be downloaded from https://drive.google.com/drive/folders/1AyKSLjt5OmTZvEvi4cp2lJt7FIvFKvzG

In [ ]:
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.P131.admin.tsv.gz'), "derived.P131.admin.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.settlement.admin1.tsv.gz'), "derived.alias.settlement.admin1.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.settlement.admin1.full.tsv.gz'), "derived.alias.settlement.admin1.full.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.settlement.admin1.iso.tsv.gz'), "derived.alias.settlement.admin1.iso.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.city.country.tsv.gz'), "derived.alias.city.country.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.city.us.tsv.gz'), "derived.alias.city.us.tsv.gz is missing from the input graph path"
assert os.path.exists(f'{os.environ["GRAPH"]}/derived.alias.settlement.tsv.gz'), "derived.alias.settlement.tsv.gz is missing from the input graph path"

In [ ]:
!$kypher -i isa \
    --match '(n1)-[]->(n2)' \
    --where 'n2 in ["Q19820110", "Q18667213", "Q51118703", "Q51118821"]' \
    --return 'n1' \
    -o $TEMP/isa.properties.remove.tsv.gz

In [ ]:
!$kypher -i $TEMP/isa.properties.remove.tsv.gz \
    --match '(n1)-[]->()' \
    --where "substr(n1,1,1)='P'" \
    --return 'distinct n1' \
    -o $TEMP/isa.properties.remove.distinct.tsv.gz

In [ ]:
kgtk("""cat -i "$TEMP"/isa.properties.remove.distinct.tsv.gz 
            -i "$TEMP"/properties.to.remove.tsv
            -o "$TEMP"/properties.remove.tsv.gz --mode=NONE""")

In [ ]:
kgtk("""cat -i $item
    -i "$GRAPH/derived.P131.admin.tsv.gz" 
    -o "$TEMP/derived.table-linker.items.tsv.gz"""")

In [ ]:
!$kypher -i "$TEMP/derived.table-linker.items.tsv.gz" --as table_linker_item \
    --match 'item: (n1)-[l {label: property}]->()' \
    --return 'distinct property as node1' \
    -o "$TEMP"/all.properties.tsv.gz

In [ ]:
kgtk("""ifnotexists -i "$TEMP"/all.properties.tsv.gz
    --filter-on "$TEMP"/properties.remove.tsv.gz 
    --input-keys node1 
    --filter-keys node1 
    --mode=NONE 
    -o "$TEMP"/final.properties.tsv.gz""")

In [ ]:
!$kypher -i claims \
    --match 'claims: (n1)-[l {label: property}]->(n2 {wikidatatype:"time"})' \
    --return 'n1 as node1, "context" as label, printf("d\"%s\":%s", kgtk_date_date(n2), property) as node2' \
    -o $TEMP/context.time.tsv.gz

In [ ]:
!zcat $TEMP/context.time.tsv.gz | head

In [ ]:
!$kypher -i table_linker_item -i label -i $TEMP/final.properties.tsv.gz --as final_props \
    --match 'item: (n1)-[l {label: property}]->(n2), label: (n2)-[]->(lab), final_props: (property)' \
    --return 'n1 as node1, "context" as label, printf("i%s:%s:%s", lower(kgtk_lqstring_text_string(lab)), property, n2) as node2' \
    --where 'n1 != n2' \
    -o "$TEMP"/context.labels.tsv.gz

In [ ]:
!zcat < "$TEMP"/context.labels.tsv.gz | head

In [ ]:
kgtk("""cat -i alias 
    -i "$GRAPH/derived.alias.settlement.admin1.tsv.gz" 
    -i "$GRAPH/derived.alias.settlement.admin1.full.tsv.gz" 
    -i "$GRAPH/derived.alias.settlement.admin1.iso.tsv.gz"
    -i "$GRAPH/derived.alias.city.country.tsv.gz" 
    -i "$GRAPH/derived.alias.city.us.tsv.gz" 
    -i "$GRAPH/derived.alias.settlement.tsv.gz" 
    -o "$TEMP/derived.location.aliases.tsv.gz"""")

In [ ]:
!$kypher -i table_linker_item -i "$TEMP/derived.location.aliases.tsv.gz" -i final_props \
    --match 'item: (n1)-[l {label: property}]->(n2), aliases: (n2)-[]->(alias), final_props: (property)' \
    --return 'n1 as node1, "context" as label, printf("i%s:%s:%s", lower(kgtk_lqstring_text_string(alias)), property, n2) as node2' \
    --where 'n1 != n2' \
    -o "$TEMP"/context.aliases.tsv.gz

In [ ]:
!zcat < "$TEMP"/context.aliases.tsv.gz | head

In [ ]:
!$kypher -i claims \
    --match 'claims: (n1)-[l {label: property}]->(n2 {wikidatatype:"external-id"})' \
    --return 'n1 as node1, "context" as label, printf("e%s:%s", n2, property) as node2' \
    -o "$TEMP"/context.external_id.tsv.gz

In [ ]:
!zcat < "$TEMP"/context.external_id.tsv.gz | head

In [ ]:
!$kypher -i claims \
    --match 'claims: (n1)-[l {label: property}]->(n2 {wikidatatype:"quantity"})' \
    --return 'n1 as node1, "context" as label, printf("q\"%s\":%s",kgtk_quantity_number(n2), property) as node2' \
    -o "$TEMP"/context.quantity.tsv.gz

In [ ]:
!zcat < "$TEMP"/context.quantity.tsv.gz | head

In [ ]:
!$kypher -i claims \
    --match 'claims: (n1)-[l {label: property}]->(n2 {wikidatatype:"monolingualtext"})' \
    --return 'n1 as node1, "context" as label, printf("m%s:%s",lower(kgtk_lqstring_text_string(n2)), property) as node2' \
    --where 'n2.kgtk_lqstring_lang = "en"' \
    -o "$TEMP"/context.monolingualtext.tsv.gz

In [ ]:
!zcat < "$TEMP"/context.monolingualtext.tsv.gz | head

In [ ]:
kgtk("""cat -i "$TEMP'/context.external_id.tsv.gz 
        "$TEMP"/context.quantity.tsv.gz 
        "$TEMP"/context.aliases.tsv.gz 
        "$TEMP"/context.labels.tsv.gz  
        "$TEMP"/context.time.tsv.gz 
        "$TEMP"/context.monolingualtext.tsv.gz 
        -o "$TEMP"/context.all.tsv.gz""")

In [ ]:
kgtk("""sort --columns node1 
         -i "$TEMP"/context.all.tsv.gz 
        -X "--parallel 8 --buffer-size 60%" 
        -o "$TEMP"/context.all.sorted.tsv.gz""")

In [ ]:
kgtk("""compact -i "$TEMP"/context.all.sorted.tsv.gz
        --mode=NONE 
        --columns node1 label
        --presorted True 
        -o "$OUT"/table_linker.qnode.property.values.tsv.gz""")

In [ ]:
!zcat < "$OUT"/table_linker.qnode.property.values.tsv.gz | head

## ComplEx Graph Embeddings

In [ ]:
kgtk("""graph-embeddings --verbose -i $item 
        -o $OUT/wikidatadwd.complEx.graph-embeddings.tsv.gz
        --retain_temporary_data True 
        --operator ComplEx 
        --workers 24 
        --log "$TEMP"/ge.complex.log 
        -T "$TEMP"
        -ot kgtk
        -e 600""")

The augmentation files required for the next cell can be downloaded from Google Drive,
https://drive.google.com/drive/u/1/folders/1qbbgjo7pddMdDvQzOSeSaL6lYwj_f5gi

In [ ]:
kgtk("""cat -i "$GRAPH"/all.tsv.gz 
        -i "$GRAPH"/metadata.property.datatypes.tsv.gz 
        -i "$GRAPH"/metadata.pagerank.undirected.tsv.gz 
        -i "$GRAPH"/metadata.pagerank.directed.tsv.gz 
        -i "$GRAPH"/augmentation.wikipedia.anchors.tsv.gz 
        -i "$GRAPH"/augmentation.wikipedia.tables.anchors.tsv.gz 
        -i "$GRAPH"/augmentation.wikipedia.redirect.tsv.gz 
        -i "$OUT"/item.property.count.compact.tsv.gz 
        -i "$OUT"/dwd_isa_class_count.compact.tsv.gz 
        -i "$OUT"/wikidatadwd.complEx.graph-embeddings.tsv.gz
        -i "$OUT"/derived.dwd_isa.tsv.gz 
        -i "$OUT"/table_linker.qnode.property.values.tsv.gz 
        -i "$GRAPH"/derived.isastar.tsv.gz 
        -o "$TEMP"/wikidata.dwd.table-linker.index.tsv.gz

In [ ]:
kgtk(f"""sort -i "$TEMP"/wikidata.dwd.table-linker.index.tsv.gz 
        --columns node1 
        --extra '--parallel 24 --buffer-size 30% --temporary-directory ' + {os.environ['TEMP']} 
        -o "$OUT"/wikidata.dwd.table-linker.index.sorted.tsv.gz""")

### Create the `metadata.property.datatypes.augmented.tsv.gz` file

In [ ]:
!curl https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv -o $TEMP/kgtk-properties.tsv

In [ ]:
!$kgtk filter -p ";data_type;" -i $TEMP/kgtk-properties.tsv -o $TEMP/kgtk-properties.datatype.tsv.gz

In [ ]:
!$kgtk cat -i $TEMP/kgtk-properties.datatype.tsv.gz $OUT/metadata.property.datatypes.tsv.gz -o $OUT/metadata.property.datatypes.augmented.tsv.gz

### Build JSON Lines Files to be loaded into ElasticSearch

In [ ]:
kgtk("""build-kgtk-search-input --input-file "$OUT"/wikidata.dwd.table-linker.index.sorted.tsv.gz
--output-file "$OUT"/wikidata.dwd.table-linker.index.sorted.jl 
--label-properties label 
--alias-properties alias 
--extra-alias-properties P1448,P1705,P1477,P1810,P742,P1449 
--description-properties description 
--pagerank-properties Pdirected_pagerank 
--mapping-file "$OUT"/wikidata_dwd.v2.table-linker.json 
--property-datatype-file "$OUT"/metadata.property.datatypes.augmented.tsv.gz""")

#### Move all the files to input folder

In [ ]:
!mv $OUT/* $GRAPH/